### Code to Import the trained model and check for prediction

Name : Rinku Diwakar

Roll No: 23126042

In [28]:
import pickle
import os
from datetime import datetime
import pandas as pd
import numpy as np

In [4]:
model_path=r"D:\Desktop\Programming\machine_learning_project\Seoul_bike_demand_forecasting\Models\xgboost_regressor_r2_0_917_v1.pkl"
model=pickle.load(open(model_path,"rb"))

In [9]:
data=[[-0.50839691,  1.1998884 ,  0.52836228,  0.06680005,  0.9210637 ,
         0.14200505, -0.13401915, -0.17102636, -0.22996655,  0.        ,
        -0.65043864,  0.43077237,  0.30179287, -0.58569899,  1.73073366,
        -0.56834235, -0.40129017, -0.41177593, -0.40678594, -0.40559398,
        -0.4113018 , -0.41059018],
      [ 0.07092757,  1.65407403, -0.35217637,  0.3562486 , -1.04340087,
         3.01478263, -0.13401915, -0.17102636, -0.22996655,  0.        ,
         0.48569306,  0.1403305 ,  0.30179287, -0.58569899,  1.73073366,
        -0.56834235, -0.40129017, -0.41177593, -0.40678594, -0.40559398,
        -0.4113018 , -0.41059018]
    
     ]
prediction=model.predict(data)
list(prediction)

[np.float32(1704.3315), np.float32(979.20465)]

### Load SC 

In [10]:
sc_dump_path=r"D:\Desktop\Programming\machine_learning_project\Seoul_bike_demand_forecasting\Models\sc.pkl"
sc=pickle.load(open(sc_dump_path,"rb"))

In [11]:
sc

StandardScaler()

## Users Input

In [100]:
date="24/04/2025"
hour=9
temperature =25
humidity =25
wind_speed = 60
visibility=60
solar_radiation=1.8
rainfall=0.7
snowfall=0.2
holiday='No Holiday'
seasons='Summer'
functioning_day='Yes'

## Converting User data into model consumable data

In [101]:
holiday_dic={"No Holiday":0,"Holiday":1}
functioning_day_dic={"NO":0,"Yes":1}
def get_string_date_time(date):
    dt=datetime.strptime(date,"%d/%m/%Y")
    return{"day":dt.day,"Month":dt.month,"year":dt.year,"weekday":dt.strftime("%A")}
str_to_date=get_string_date_time(date)
str_to_date

{'day': 24, 'Month': 4, 'year': 2025, 'weekday': 'Thursday'}

In [102]:
u_input_list=[hour,temperature,humidity,wind_speed,visibility,solar_radiation,rainfall,snowfall,holiday_dic[holiday],functioning_day_dic[functioning_day],
             str_to_date["day"],str_to_date["Month"],str_to_date["year"]]

features_name=['Hour', 'Temperature(°C)', 'Humidity(%)',
       'Wind speed (m/s)', 'Visibility (10m)', 'Solar Radiation (MJ/m2)',
       'Rainfall(mm)', 'Snowfall (cm)', 'Holiday', 'Functioning Day', 'Day',
       'Month', 'Year']
df_u_input=pd.DataFrame([u_input_list],columns=features_name)
df_u_input

,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Holiday,Functioning Day,Day,Month,Year
0,9,25,25,60,60,1.8,0.7,0.2,0,1,24,4,2025


In [103]:
u_input_list

[9, 25, 25, 60, 60, 1.8, 0.7, 0.2, 0, 1, 24, 4, 2025]

In [104]:
def season_to_df(seasons):
    seasons_cols=['Spring', 'Summer', 'Winter']
    seasons_data=np.zeros((1,len(seasons_cols)),dtype="int")
    df_seasons=pd.DataFrame(seasons_data,columns=seasons_cols)
    if seasons in seasons_cols:
        df_seasons[seasons]=1
    return df_seasons
season_df=season_to_df(seasons)
season_df

,Spring,Summer,Winter
0,0,1,0


In [105]:
def days_to_df(weekday):
    days_name=['Monday', 'Saturday',
       'Sunday', 'Thursday', 'Tuesday', 'Wednesday']
    days_data=np.zeros((1,len(days_name)),dtype="int")
    df_days=pd.DataFrame(days_data,columns=days_name)
    if weekday in days_name:
        df_days[weekday]=1
    return df_days
days_df=days_to_df(str_to_date["weekday"])  
days_df


,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,0,0,0,1,0,0


In [106]:
df_for_pred=pd.concat([df_u_input,season_df,days_df],axis=1)
df_for_pred

,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Holiday,Functioning Day,...,Year,Spring,Summer,Winter,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,9,25,25,60,60,1.8,0.7,0.2,0,1,...,2025,0,1,0,0,0,0,1,0,0


In [107]:
sc_data_for_pred=sc.transform(df_for_pred)
sc_data_for_pred

array([[-0.36356579,  1.01484981, -1.62406554, 56.21981749, -2.25583104,
         1.40099124,  0.52198199,  0.27197964, -0.22996655,  0.        ,
         0.94014573, -0.7309951 , 25.60905941, -0.58569899,  1.73073366,
        -0.56834235, -0.40129017, -0.41177593, -0.40678594,  2.4655198 ,
        -0.4113018 , -0.41059018]])

## Prediction in Deployment

In [108]:
print(f"Rented bike demand on date : {date}, and Time : {hour} hour is : {round(model.predict(sc_data_for_pred).tolist()[0])}")

Rented bike demand on date : 24/04/2025, and Time : 9 hour is : 663
